Here we should make sure that our Transformer implementation works properly. For this purpose we gonna learn to translate on En-De pair

Let's tokenize our sentences (TODO: should we use `moses` for this and not `nltk` because moses can normalize punctuation?)

In [24]:
# %%bash

# cat ../data/multi30k/val.en | \
#     ../ext-libs/mosesdecoder/scripts/tokenizer/normalize-punctuation.perl | \
#     ../ext-libs/mosesdecoder/scripts/tokenizer/tokenizer.perl -threads 1 > \
#     ../data/generated/kek
    
# head ../data/generated/kek

In [31]:
import os

input_data_dir = '../data/multi30k'
output_data_dir = '../data/generated'

if not os.path.exists(output_data_dir): os.mkdir(output_data_dir)

In [28]:
import nltk
nltk.download('punkt')

for file_name in os.listdir(input_data_dir):
    input_file_path = '{}/{}'.format(input_data_dir, file_name)
    output_file_path = '{}/{}.tok'.format(output_data_dir, file_name)
    
    with open(input_file_path, 'r', encoding='utf-8') as file:
        lines = file.read().splitlines()
    
    tokenized = [' '.join(nltk.word_tokenize(line)) for line in lines]
    
    with open(output_file_path, 'w', encoding='utf-8') as file:
        for line in tokenized:
            file.write(line + os.linesep)

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/universome/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Generating BPEs (joint for both langs)

In [30]:
%%bash
src="../data/generated/train.en.tok"
trg="../data/generated/train.de.tok"
num_bpes=16000

bpes="../data/generated/bpes"
src_vocab="../data/generated/vocab.en"
trg_vocab="../data/generated/vocab.de"

python ../ext-libs/subword-nmt/learn_joint_bpe_and_vocab.py \
    --input "$src" "$trg" \
    -s "$num_bpes" \
    -o "$bpes" \
    --write-vocabulary "$src_vocab" "$trg_vocab"

# Now we should apply bpe for all our train/test/val files
for file in $(ls ../data/multi30k)
do
    vocab="../data/generated/vocab.${file: -2}"
  
    python ../ext-libs/subword-nmt/apply_bpe.py -c $bpes \
       --vocabulary "$vocab" \
       < "../data/multi30k/$file" > "../data/generated/$file.bpe"
done

Ok, now we have nice dataset, which we can use for training.
So let's train our Transformer!